In [14]:
from sklearn.metrics.pairwise import rbf_kernel
import numpy as np
from scipy import sparse 
import sys
from gaussiandata import SyntheticTarget
from online_prototypes import greedy_select_protos
from online_criticism import greedy_select_criticism_online
#import greedy_functions
import math
import scipy

In [13]:
SEED = 666
number_of_gaussians = 2
ex = SyntheticTarget(seed=SEED, dist_components=number_of_gaussians)

N = 10000
M = 100
data = ex.draw_sample(N)
print data[(2,3,4),:]
gamma = 2
kernel_matrix = rbf_kernel(data, gamma = gamma)

[[ 3.64703263 -3.77549178]
 [-0.42829885 -3.19168078]
 [ 3.175412   -3.63978629]]


In [ ]:
prototype_indices = greedy_select_protos(kernel_matrix, candidate_indices = np.array(range(N)), m = M)

In [ ]:
print(prototype_indices)

In [ ]:
def rbf(x,y, gamma):
    x = np.asarray(x)
    y = np.asarray(y)
    return scipy.exp(-gamma*np.linalg.norm(x-y))

def our_rbf_kernel(current_kernel_subset,  gamma, x_new, n, timeseries):
    new_row = np.zeros(n)
    for i in range(n):
         new_row[i] = rbf(x = x_new, y = timeseries[i], gamma = gamma)
    new_kernel_subset = np.vstack((current_kernel_subset, new_row))
    return new_kernel_subset

In [ ]:
gamma = 2
prototype_indices = np.arange(M) #assign first M points as prototypes. CONSIDER another method later.
criticism_indices = np.arange(M, 2*M) # assign second M points as prototypes.

#calculate the initial kernel matrix for the first M points
current_kernel_matrix_proto = rbf_kernel(data[prototype_indices,:], gamma=gamma)

#current_kernel_matrix_criticism = rbf_kernel(data[np.array(prototype_indices, criticism_indices),:], gamma = gamma)

for i in range(2*M, N):
    new_point_index = i # assign the new data point index
    
    accept == 1 # decide whether to skip the criticism part or not 
    
    #prototype part
    if accept == 1:
        #add new column and row to the kernel matrix for the new point; 
        #kernel matrix is always growing
        #current_kernel_subset,  gamma, x_new, n, timeseries):
        new_kernel_matrix_proto = our_rbf_kernel(gamma = gamma, current_kernel_subset = current_kernel_matrix_proto, n = i, timeseries = timeseries, x_new= x_new)
        
        #run the greedy algorithm on these M+1 data points to select M prototypes
        prototype_indices_temp = greedy_select_protos(new_kernel_matrix_proto, 
                                                  candidate_indices = np.vstack((prototype_indices, i-1)), m = M)
    
        if prototype_indices_temp == prototype_indices:
            accept == 0
        
        else:
            prototype_indices = prototype_indices_temp # update the indices
            current_kernel_matrix_proto = new_kernel_matrix_proto # update the kernel matrix
    
    
    #criticism part
    if accept == 0:
        
    
    

In [ ]:
np.arange(M+1)

In [ ]:
current_kernel_subset =  np.random.randn(5,10)
our_rbf_kernel(current_kernel_subset = current_kernel_subset, gamma = 2, x_new = 6, n = 10, timeseries = np.arange(10))


In [ ]:
new_row = np.zeros(10)

In [ ]:
print new_row

In [ ]:
np.arange(10)

In [ ]:
A =  np.random.randn(5,10)
B =  np.random.randn(5,10)
